In [45]:
# import niezbednych pakietów
import pandas as pd
import scipy
import numpy as np
from sentence_transformers import SentenceTransformer
from langdetect.lang_detect_exception import LangDetectException
from langdetect import detect

In [46]:
# wczytanie danych
tweets = pd.read_csv('tweets.csv')

In [47]:
tweets.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [48]:
# tworzę nową tabelę z wartościami w kolumnie airline_sentiment_confidence =1, aby mieć pewność co do przypisania
# twittów. 
tweets_conf = tweets[tweets['airline_sentiment_confidence'] == 1]

In [49]:
#sprawdzenie skuteczności wykonania operacji
tweets_conf['airline_sentiment_confidence'].value_counts()

1.0    10445
Name: airline_sentiment_confidence, dtype: int64

In [50]:
# sprawdzenie w jakim języku napisany jest tweet
for index, row in tweets_conf['text'].iteritems():
    try:
        language = detect(row) #detecting each row
        tweets_conf.loc[index, 'language'] = language
    except LangDetectException:
        pass 

/Users/mateuszmaterek/Documents/envs/dummy/lib/python3.7/site-packages/pandas/core/indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/mateuszmaterek/Documents/envs/dummy/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [51]:
#sprawdzenie jakie języki zostały wykryte
tweets_conf['language'].value_counts()

en    10341
fr       32
da       16
af       15
it        8
so        7
cy        7
ro        4
de        4
tl        2
nl        2
tr        2
sv        1
et        1
id        1
es        1
no        1
Name: language, dtype: int64

In [52]:
# utworzenie tabeli wylacznie z tweetami w jez angielskim
tweets_conf = tweets_conf[(tweets_conf['language'].str.contains("en") == True)];

In [53]:
tweets_conf.shape

(10341, 16)

In [54]:
# swtorzenie nowej tabeli z informacją czy opinia jest negatywna, czy nie (pozytywne i neutralne łącze 
# do jednej kategorii)
def opinion(row):
    if (row['airline_sentiment'] == 'positive' or row['airline_sentiment'] == 'neutral'):
        return 0
    else:
        return 1

tweets_conf['Opinion'] = tweets_conf.apply(opinion, axis=1)  

In [55]:
tweets_conf.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,language,Opinion
0,570306133677760513,neutral,1.0,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada),en,0
3,570301031407624196,negative,1.0,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada),en,1
4,570300817074462722,negative,1.0,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada),en,1
5,570300767074181121,negative,1.0,Can't Tell,0.6842,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica seriously would pay $30 a fligh...,NaN,2015-02-24 11:14:33 -0800,NaN,Pacific Time (US & Canada),en,1
9,570295459631263746,positive,1.0,NaN,NaN,Virgin America,NaN,YupitsTate,NaN,0,"@VirginAmerica it was amazing, and arrived an ...",NaN,2015-02-24 10:53:27 -0800,Los Angeles,Eastern Time (US & Canada),en,0


In [56]:
# usunięcie zbdnych kolumn
tweets_conf = tweets_conf.drop(tweets_conf.columns[:10],axis=1)
tweets_conf = tweets_conf.drop(tweets_conf.columns[[1,2,3,4,5]],axis=1)
tweets_conf.head()

,text,Opinion
0,@VirginAmerica What @dhepburn said.,0
3,@VirginAmerica it's really aggressive to blast...,1
4,@VirginAmerica and it's a really big bad thing...,1
5,@VirginAmerica seriously would pay $30 a fligh...,1
9,"@VirginAmerica it was amazing, and arrived an ...",0


In [57]:
tweets_conf.shape

(10341, 2)

In [58]:
import re
# zmieniam wielkość wszystkich liter na małe
tweets_conf['text'] = tweets_conf['text'].str.lower()

# usuwam słowa zaczynające sie na @
tweets_conf['text'] = tweets_conf['text'].map(lambda x:re.sub('@\w*','',str(x)))

tweets_conf.head()

,text,Opinion
0,what said.,0
3,"it's really aggressive to blast obnoxious ""en...",1
4,and it's a really big bad thing about it,1
5,seriously would pay $30 a flight for seats th...,1
9,"it was amazing, and arrived an hour early. yo...",0


In [59]:
# wyodrębnienie słów kluczowych tweetów
import yake
def key_words_extractor(row):
    kw_extractor = yake.KeywordExtractor()
    keywords = kw_extractor.extract_keywords(row['text'])
    ka = [x[1] for x in keywords]
    ka = [item.replace("'",'').replace("'",'') for item in ka]
    ka = str(ka)[1:-1]
    return ka

tweets_conf['keywords'] = tweets_conf.apply(key_words_extractor, axis=1)  

/Users/mateuszmaterek/Documents/envs/dummy/lib/python3.7/site-packages/yake/datarepresentation.py:121: RuntimeWarning: Mean of empty slice.
  avgTF = validTFs.mean()
/Users/mateuszmaterek/Documents/envs/dummy/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/mateuszmaterek/Documents/envs/dummy/lib/python3.7/site-packages/numpy/core/_methods.py:217: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/Users/mateuszmaterek/Documents/envs/dummy/lib/python3.7/site-packages/numpy/core/_methods.py:186: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/Users/mateuszmaterek/Documents/envs/dummy/lib/python3.7/site-packages/numpy/core/_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [62]:
tweets_conf.sample(10)

,text,Opinion,keywords
10883,is a joke. stop losing my luggage!,1,"'joke', 'stop', 'luggage', 'losing', 'stop los..."
9465,i am in meetings in california and have no lu...,1,"'meetings in california', 'luggage', 'meetings..."
9841,sitting at the gate on flight 719 for an hour...,1,"'sitting', 'gate on flight', 'sabre', 'hour du..."
11559,"lack of communication with other offices, the...",1,"'customer', 'lack', 'offices', 'yall', 'system..."
10110,f623 den:phx impressive lack of concern by 1h...,1,"'late flight flight', 'den', 'smugsmirk', 'nor..."
10507,looking forward to waiting on hold for 9 hrs ...,1,"'reschedule my flight', 'forward to waiting', ..."
1844,hi yes need a free hotel stay in honalulu due...,1,"'flight to kauai', 'free hotel stay', 'hour', ..."
4224,i don't need to make any changes. i reserved ...,1,"'seats', 'make', 'error', 'reserved', 'comp', ..."
922,searched for mileage tickets to bom couldn't ...,1,"'saver awards', 'searched for mileage', 'searc..."
11746,"told no gates open at #ord, needed to wait 20...",1,"'ord', 'needed to wait', 'told', 'needed', 'wa..."


In [63]:
# podział danych na zbiory testowe i treningowe
tweets_conf_train = tweets_conf[:7000]
tweets_conf_test = tweets_conf[7000:]

In [64]:
# wyodrebnienie list z negatywnymi i pozytywnymi opiniami
positive = tweets_conf_train[tweets_conf_train['Opinion'] == 0]
negative = tweets_conf_train[tweets_conf_train['Opinion'] == 1]

keywords_positive = positive["keywords"].tolist()
keywords_negative = negative["keywords"].tolist()

In [65]:
# wygenerowanie wektorów embeddingu dla opinii pozytywnych i negatywnych
embedder = SentenceTransformer('bert-base-nli-mean-tokens')

negative_embeddings = embedder.encode(keywords_negative)
positive_embeddings = embedder.encode(keywords_positive)

In [66]:
# rozpoznawanie tweetow negatywnych i pozytywnych poprzez porownanie podobienstwa cosinusowego emedingu nowej 
# wiadomosci z embedingami opinii pozytywnych i negatywnych. Jesli srednie podobienstwo do tweetow pozytywnych jest 
# wieksze niz dla negatywnych to opinia zostaje uznana na pozytywna.

def recognition(new_comment):
    
    corpus_embeddings = embedder.encode([new_comment])
    
    positive_list = []
    for i in positive_embeddings:
        p_similarity = -scipy.spatial.distance.cosine(corpus_embeddings, i) + 1
        positive_list.append(p_similarity)
    mean_p_sim = np.mean(positive_list)
    
    negative_list = []
    for i in negative_embeddings:
        n_similarity = -scipy.spatial.distance.cosine(corpus_embeddings, i) + 1
        negative_list.append(n_similarity)
    mean_n_sim = np.mean(negative_list)
    
    if mean_p_sim > mean_n_sim:
        answer = 'postive'
    else:
        answer = 'negative'
        
    return answer
    

In [67]:
# sprawdzmy dzialanie na przykladowych wpisow
recognition('excellent travel')

'postive'

In [68]:
recognition('where is my money?')

'negative'

In [69]:
recognition('the worst airline')

'negative'

In [70]:
recognition('good lunch, helpful staff')

'postive'

In [72]:
# funkcja rozpoznajaca opinie ze zbioru testowego na podstawie embeddingow ze zbioru treningowego

def test_recognition(row):
    corpus_embeddings = embedder.encode([row['keywords']])
    
    positive_list = []
    for i in positive_embeddings:
        p_similarity = -scipy.spatial.distance.cosine(corpus_embeddings, i) + 1
        positive_list.append(p_similarity)
    mean_p_sim = np.mean(positive_list)
    
    negative_list = []
    for i in negative_embeddings:
        n_similarity = -scipy.spatial.distance.cosine(corpus_embeddings, i) + 1
        negative_list.append(n_similarity)
    mean_n_sim = np.mean(negative_list)
    
    if mean_p_sim > mean_n_sim:
        answer = 0
    else:
        answer = 1
        
    return answer

tweets_conf_test['recognition'] = tweets_conf_test.apply(test_recognition, axis=1) 

/Users/mateuszmaterek/Documents/envs/dummy/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [87]:
# sprawdzenie przykladowych wynikow
tweets_conf_test.sample(10)

,text,Opinion,keywords,recognition
11878,how's us 1797 looking today?,0,'today',0
14324,complt incompetence on flt 295.lav delay from...,1,"'complt incompetence', 'poor svc', 'lnded last...",1
13053,"thanks gang! mind if i ask, do you handle all...",0,"'gang', 'mind', 'house', 'handle', 'social', '...",0
13292,"has the worst flights and customer service, t...",1,"'customer service', 'cancelled flighted', 'fli...",1
13127,"is that the ""record locator""?",0,"'record locator', 'record', 'locator'",0
13784,"cost me a day of vacation ($350 hotel stay),...",1,"'hotel stay', 'dog boarding', 'airport parking...",1
13216,i totally agree. you get the automatic phon...,1,"'totally agree', 'agree', 'totally', 'service'...",0
12266,fly my child,0,"'fly my child', 'fly', 'child'",0
13368,record locator oaaret got email we couldn't b...,1,"'record locator oaaret', 'record locator', 'ca...",1
12440,just need to know why it wasn't done like it ...,1,"'yesterday', 'promised'",0


In [88]:
def score(row):
    if (row['recognition'] == row['Opinion']):
        return 1
    else:
        return 0

tweets_conf_test['opinion vs recognition'] = tweets_conf_test.apply(assess, axis=1) 

/Users/mateuszmaterek/Documents/envs/dummy/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [89]:
tweets_conf_test.sample(10)

,text,Opinion,keywords,recognition,opinion vs recognition
11767,flight 1808 is not leaving at 2:45 because we...,1,"'flight', 'board', 'leaving', '2:45', 'begun',...",1,1
13362,i did set up notifications through aa &amp; u...,1,"'airways', 'set', 'notifications', 'set up not...",0,0
10780,why i can't book a flight to prn through your...,1,"'website', 'book', 'flight', 'prn', 'book a fl...",1,1
10891,"now i am on flight to fll, and told to take a...",1,"'flight to fll', 'back to pbi', 'train back', ...",1,1
13889,"jal and cathay pacific so no, i can't do it o...",1,"'jal and cathay', 'jal', 'online', 'cathay', '...",0,0
11390,stop cancelled flighting flights over and ove...,1,"'stop cancelled flighting', 'stop cancelled', ...",1,1
11436,you are the best!!! follow me please;)🙏🙏🙏✌️✌️...,0,'follow',0,1
14419,two hours late flightr and still waiting for ...,1,"'callback', 'late', 'flightr', 'hours', 'waiti...",1,1
12399,i did not get any compensation after cancell...,1,"'compensation after cancelled', 'cancelled fli...",1,1
13877,i was worried when my bag went san to dca 24 ...,1,"'organized breeze', 'san to dca', 'breeze', 'b...",1,1


In [94]:
# ocena wyników  (procent poprawnych rozpoznań)
tweets_conf_test['opinion vs recognition'].sum()/tweets_conf_test.shape[0]

0.8323855133193655